In [23]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QCheckBox, QSpinBox
import serial
import time
import base64
#import keyboard

# UI 

In [24]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton, QVBoxLayout, QLabel, QCheckBox, QSpinBox

class MotorControlerUI(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()
        self.ui_events = []

    def initUI(self):
        # Create buttons s1 to s6
        self.buttons = []
        button_pos =[(50,50),(50,100),(50,150),(50,200),(50,250),(50,300)]
        for i in range(1, 7):
            button = QPushButton(f'S{i}', self)
            button.clicked.connect(self.buttonClicked)
            button.move(button_pos[i-1][0],button_pos[i-1][1])
            self.buttons.append(button)

        #labels for s1->s6
        self.labels = []
        label_pos = [(bp[0]*4,bp[1]) for bp in button_pos]
        for i in range(1,7):
            label = QLabel(f"Label{i}",self)
            label.move(label_pos[i-1][0],label_pos[i-1][1])
            label.setText("0")
            label.setFixedWidth(50)
            self.labels.append(label)

        #spinBox for s1->s6
        self.spinboxs = []
        spinbox_pos = [(bp[0]*6,bp[1]) for bp in button_pos]
        for i in range(1,7):
            spinbox = QSpinBox(self)
            spinbox.setObjectName(f"s{i}")
            spinbox.setRange(-500,500)
            spinbox.setFixedWidth(150)
            spinbox.move(spinbox_pos[i-1][0],spinbox_pos[i-1][1])
            self.spinboxs.append(spinbox)
        
        #buton pairs (1-6,2-5,3-4)
        self.pair_buttons = []
        pair_button_pos = [(bp[0]*10,bp[1]) for bp in button_pos[:3]]
        pair_button_names = ["S1-6","S2-5","S3-4"]
        for i in range(3):
            button = QPushButton(pair_button_names[i], self)
            button.clicked.connect(self.pairButtonCliked)
            button.move(pair_button_pos[i][0],pair_button_pos[i][1])
            self.pair_buttons.append(button)
        
        #spinbox pairs (1-6,2-5,3-4)
        self.pair_spinboxs = []
        pair_spinbox_pos = [(bp[0]*12,bp[1]) for bp in button_pos[:3]]
        for i in range(3):
            spinbox = QSpinBox(self)
            spinbox.setObjectName(pair_button_names[i])
            spinbox.setRange(-500,500)
            spinbox.setFixedWidth(150)
            spinbox.move(pair_spinbox_pos[i][0],pair_spinbox_pos[i][1])
            self.pair_spinboxs.append(spinbox)


        #time input
        self.timeInput = QSpinBox(self)
        self.timeInput.setObjectName("timeInput")
        self.timeInput.setRange(800,100000)
        self.timeInput.setFixedWidth(150)
        self.timeInput.move(spinbox_pos[5][0],spinbox_pos[5][1]+100)

        #time input explanation
        self.labelTime = QLabel("Time pr. instruction",self)
        self.labelTime.move(label_pos[5][0]-50,label_pos[5][1]+100)

        #spinbox header
        self.labelInfo = QLabel("- = ClockWise",self)
        self.labelInfo.move(spinbox_pos[0][0],spinbox_pos[0][1]-25)

        #label header
        self.labelPos = QLabel("MotorPosition",self)
        self.labelPos.move(label_pos[0][0],label_pos[0][1]-25)

        #all motors input
        self.allButton = QPushButton(f'S1-6', self)
        self.allButton.clicked.connect(self.allButtonClicked)
        self.allButton.move(button_pos[5][0],button_pos[5][1]+100)

        #start protocol
        self.startProt = QPushButton("StartProtocol",self)
        self.startProt.clicked.connect(self.startProtocol)
        self.startProt.move(button_pos[5][0],button_pos[5][1]+200)

        self.setWindowTitle('Motor controler')
        self.setGeometry(300, 300, 800, 600)  # Set window position and size

    #UPDATES
    def updateLabels(self,mc): #pass array of motor change values to update labels
        for i in range(6):
            label = self.labels[i]
            cur_val = int(label.text())
            label.setText(str(cur_val+mc[i]))


    #UI EVENTS
    def buttonClicked(self): #button click (adds spinbox to label)
        sender = self.sender()
        index = int(sender.text()[1]) - 1
        spinbox = self.spinboxs[index]
        change_val = spinbox.value()
        label = self.labels[index]
        cur_val = int(label.text())
        new_val = change_val+cur_val
        label.setText(str(new_val))
        #get time
        time_val = self.timeInput.value()

        self.ui_events.append(["1B",index,change_val,time_val,f'Button {sender.text()} clicked. With Value {change_val}: {cur_val} -> {new_val}'])

    def allButtonClicked(self): #button for running all motors at the same time
        mc = [0,0,0,0,0,0]
        time_val = self.timeInput.value()
        
        for i in range(6): #get value from each of the spin boxes
            mc[i] = self.spinboxs[i].value()
            label = self.labels[i]
            label.setText(str(mc[i]+int(label.text())))

        self.ui_events.append(["AB",mc,time_val,f"Allbutton {self.sender().text()} clicked. With Values {mc}'"])

    def startProtocol(self):
        self.ui_events.append(["SP",f"StartProtocol {self.sender().text()} clicked."])

    def pairButtonCliked(self):
        sender = self.sender().text()
        time_val = self.timeInput.value()
        mc = [0,0,0,0,0,0]
        match sender: #could be made cleaner with a map or somting
            case "S1-6":
                val = self.pair_spinboxs[0].value()
                mc[0] = val
                mc[5] = val
            case "S2-5": #s2-5
                val = self.pair_spinboxs[1].value()
                mc[1] = val
                mc[4] = val   
            case "S3-4": #s3-4
                val = self.pair_spinboxs[2].value()
                mc[2] = val
                mc[3] = val

        self.updateLabels(mc)
        self.ui_events.append(["PB",mc,time_val,f"Pairbutton {sender} clicked. With Values {mc}'"])


#Handle events
def handleUIEvents(ui_widget,mc,timeval,protocol):
    while ui_widget.ui_events:
        event = ui_widget.ui_events.pop(0)
        print(event)

        match event[0]: #TODO: make a standard structure for events (but good enough for now)
            case "1B": #1:idx 2:changeVal 3: time val
                mc[event[1]] = event[2]
                timeval[0] = event[3]
            case "AB": #1: mcVal #2:time val
                #print(mc)
                for i in range(6):
                    mc[i] = event[1][i]
                #mc = event[1]
                #print(mc)
                timeval[0] = event[2]
            case "SP":
                protocol[0] = True
            case "PB":
                for i in range(6):
                    mc[i] = event[1][i]
                timeval[0] = event[2]
            case _:
                print(f"Unknown first event {event[0]}")

    app.processEvents()


# Serial protocol

In [25]:
#initial protocol from https://forum.arduino.cc/t/serial-input-basics-updated/382007/2
# and https://forum.arduino.cc/t/pc-arduino-comms-using-python-updated/574496 

startMarker = '<'
endMarker = '>'
dataStarted = False
dataBuf = ""
messageComplete = False

#========================
#========================
    # the functions

def setupSerial(baudRate, serialPortName):
    global  serialPort
    serialPort = serial.Serial(port= serialPortName, baudrate = baudRate, timeout=0, rtscts=True)
    print("Serial port " + serialPortName + " opened  Baudrate " + str(baudRate))
    waitForArduino()

#========================

def sendToArduino(stringToSend):
    # this adds the start- and end-markers before sending
    global startMarker, endMarker, serialPort
    
    stringWithMarkers = (startMarker)
    stringWithMarkers += stringToSend
    stringWithMarkers += (endMarker)

    serialPort.write(stringWithMarkers.encode('utf-8')) # encode needed for Python3


#==================

def recvLikeArduino():

    global startMarker, endMarker, serialPort, dataStarted, dataBuf, messageComplete

    if serialPort.inWaiting() > 0 and messageComplete == False:
        x = serialPort.read().decode("utf-8") # decode needed for Python3
        
        if dataStarted == True:
            if x != endMarker:
                dataBuf = dataBuf + x
            else:
                dataStarted = False
                messageComplete = True
        elif x == startMarker:
            dataBuf = ''
            dataStarted = True
    
    if (messageComplete == True):
        messageComplete = False
        return dataBuf
    else:
        return "XXX" 

#==================

def waitForArduino():
    # wait until the Arduino sends 'Arduino is ready' - allows time for Arduino reset
    # it also ensures that any bytes left over from a previous message are discarded
    print("Waiting for Arduino to reset")
     
    msg = ""
    while msg.find("Arduino is ready") == -1:
        msg = recvLikeArduino()
        if not (msg == 'XXX'): 
            print(msg)

# Messaging protocol

In [26]:
#<msg> (where "<" starts a msg and ">" ends 1)

#start <
#first char is used for identifying the type of msg. M=reltive move, R=reset
#next 4x6 chars are used for the amount you want the steppers to move 2signed bytes pr stepper
#4bytes for the amount of time to complete the moves
#end >

def num_to_hex(num):
  if num > 32767  or num < -32768:
    raise TypeError(f"Number {num} is does not fit withing 4 char hex")
  
  # Convert negative numbers to two's complement representation
  if num < 0:
    num = (1 << 16) + num
  
  hex_value = hex(num & 0xFFFF)[2:].zfill(4)  # Convert to hexadecimal and pad with zeros
  return hex_value

def hex_to_num(hex_string):
  if not isinstance(hex_string, str) or len(hex_string) != 4:
    TypeError(f"Input must be a string of 4 characters")
  
  num = int(hex_string, 16)
  # Handle negative numbers
  if num >= 0x8000:
      num -= 1 << 16
  return num

#convert array of relative motor steps to a string that can be sent to the arduino
#current types: (R) and M
def encode_motor_msg(motor_steps,timer,msg_type):
  if not(msg_type in ["M","R"]):
    TypeError(f"Unknown msg_type: {msg_type}")
  
  if len(motor_steps) != 6:
    TypeError(f"Expected 6 motor values got: {motor_steps}")
  
  motor_hex = ""
  for ms in motor_steps:
    motor_hex += num_to_hex(ms)
  
  time_hex = num_to_hex(timer)

  return msg_type + motor_hex + time_hex

#decode motor message

def decode_motor_msg(motor_msg):
  msg_type = motor_msg[0]
  motor_hex = motor_msg[1:25]
  timer_hex = motor_msg[25:29]

  motor_vals = []
  for i in range(0,24,4):
    motor_vals.append(hex_to_num(motor_hex[i:i+4])) 
  

  return msg_type, motor_vals, hex_to_num(timer_hex)


# Examples:
numbers = [-1234, 32767, 9012, 3456, -7890, 1234]

for i in numbers:
  hexval = num_to_hex(i)
  print(hexval,hex_to_num(hexval))

fullEncode = encode_motor_msg(numbers,1000,"M")
print(fullEncode,decode_motor_msg(fullEncode))

fb2e -1234
7fff 32767
2334 9012
0d80 3456
e12e -7890
04d2 1234
Mfb2e7fff23340d80e12e04d203e8 ('M', [-1234, 32767, 9012, 3456, -7890, 1234], 1000)


# Protocols

In [27]:
import itertools

# Define the ranges for each number
ranges = [
    range(-40, 41,40),  # Range for a
    range(-40, 41,40),  # Range for b
    range(-40, 41,40),  # Range for c
    range(-40, 41,40),  # Range for d
    range(-40, 41,40),  # Range for e
    range(-40, 41,40),  # Range for f
]

# Generate all combinations
combinations = list(itertools.product(*ranges))
#print(len(combinations))
comb_list = list(map(list,combinations))
#print(combinations)
#print(len(set(combinations)))
#print(set(combinations))

#print(comb_list)

In [28]:
#function for repeatin protocol (does not create copies, uses same ref)
def repeatProtocol(prot,amount):
  return prot*amount

a = [[1],[2],[3]]
repeatProtocol(a,4)

def calcRelativePos(cur,moves):
  rel_moves = []
  cur_poses = []
  for m in moves:
    rel = [0,0,0,0,0,0]
    for i in range(6):
      rel[i] = m[i] - cur[i]
    cur = m
    rel_moves.append(rel)
    cur_poses.append(cur)
  return rel_moves, cur_poses

In [29]:
prot_60data, _ = calcRelativePos([0,0,0,0,0,0],comb_list)
prot_60data_50 = prot_60data[:50]

print(len(prot_60data),len(prot_60data_50))


729 50


In [30]:
 #up -> reset
prot_ur = [[100,100,100,100,100,100],[-100,-100,-100,-100,-100,-100]]
#up->reset->donw->reset
prot_full = [[100,100,100,100,100,100],[-100,-100,-100,-100,-100,-100],[-100,-100,-100,-100,-100,-100],[100,100,100,100,100,100]] 
prot_34 = [[0,0,100,100,0,0],[0,0,-100,-100,0,0],[0,0,-100,-100,0,0],[0,0,100,100,0,0]]

prot_dr = [[-100,-100,-100,-100,-100,-100],[100,100,100,100,100,100]]

prot_34_down = [[0,0,100,100,0,0],[0,0,-100,-100,0,0]]


prot_full_10 = repeatProtocol(prot_full,10)
prot_updwon_10 = repeatProtocol(prot_ur,50)
prot_34_10 = repeatProtocol(prot_34,10)
prot_dr_10 = repeatProtocol(prot_dr,50)

prot_34_down_50 = repeatProtocol(prot_34_down,50)

In [31]:
prot_60data[411]

[0, 0, 0, 0, 40, -80]

In [32]:
#0/0

# Main loop


In [33]:
#setupSerial(115200, "COM3")
setupSerial(1000_000, "COM4")
count = 0
prevTime = time.time()

motor_vals = [0,0,0,0,0,0] #current motor positions
motor_change = [0,0,0,0,0,0] #Changes for next motor iteration
time_val = [1] # time pointer

def calcMotorvals(cur,new):
    return [a+b for a,b in zip(cur,new)]

protocol_running = [False] #protocol pointer
protocol_ctr = 0 #counter for where in protocl we are
protocol_delay = 0.2 #in seconds
protocol_mc = prot_60data#prot_updwon_10 #prot_dr_10
protocol_prevtime = 0
protocol_maxctr = len(protocol_mc)


if __name__ == '__main__':
    app = QApplication(sys.argv)
    mcUI = MotorControlerUI()
    mcUI.show()

    # Your main loop
    while True:
        # Receive and handle messages from the microcontroller 
        arduinoReply = recvLikeArduino() # check for a reply
        if not (arduinoReply == 'XXX'):
            print ("Time %s  Reply %s" %(time.time(), arduinoReply))
    
        # Handle UI events
        handleUIEvents(mcUI,motor_change,time_val,protocol_running)

        # Respond to the microcontroller
        if motor_change != [0,0,0,0,0,0] and not protocol_running[0]: #control with UI
            motor_vals = calcMotorvals(motor_vals,motor_change) #update total motor values
            enc_msg = encode_motor_msg(motor_change,time_val[0],"M")
            sendToArduino(enc_msg)
            print("motorchange:",motor_change,"msg:", enc_msg, "time:", time_val)

            motor_change = [0,0,0,0,0,0]

        elif protocol_running[0]:
            if time.time() - protocol_prevtime > protocol_delay: #send next move after certain time
                protocol_prevtime = time.time()
                motor_vals = calcMotorvals(motor_vals,protocol_mc[protocol_ctr])

                mcUI.updateLabels(protocol_mc[protocol_ctr]) #update labels in UI

                enc_msg = encode_motor_msg(protocol_mc[protocol_ctr],int(protocol_delay*1000),"M")
                print(f"pyt: encoded msg: {enc_msg}")
                sendToArduino(enc_msg)

                print(protocol_prevtime,f"protocol iteration {protocol_ctr}",protocol_mc[protocol_ctr])
                protocol_ctr += 1
            
            if protocol_ctr == protocol_maxctr: #reset after protocol is finished
                protocol_running[0] = False
                protocol_ctr = 0
                print("protocol finished")


        #elif time.time() - prevTime > 1.0: # send a message at intervals
        #    sendToArduino("this is a test " + str(count))
        #    prevTime = time.time()
        #    count += 1
    
    sys.exit(app.exec_()) #independently run the program


Serial port COM4 opened  Baudrate 1000000
Waiting for Arduino to reset


Arduino is ready
['SP', 'StartProtocol StartProtocol clicked.']
1713798928.9635093 protocol iteration 0 [-40, -40, -40, -40, -40, -40]
Time 1713798928.9655087  Reply first: M
Time 1713798928.9745111  Reply This just in ... Mffd8ffd8ffd8ffd8ffd8ffd800c8   M   -40   -40   -40   -40   -40   -40   200   8626
Time 1713798928.9755125  Reply Start driving8627
1713798929.163523 protocol iteration 1 [0, 0, 0, 0, 0, 40]
Time 1713798929.171526  Reply Finished driving 8827
Time 1713798929.171526  Reply first: M
Time 1713798929.1755254  Reply This just in ... M00000000000000000000002800c8   M   0   0   0   0   0   40   200   8829
Time 1713798929.1765258  Reply Start driving8830
1713798929.3641467 protocol iteration 2 [0, 0, 0, 0, 0, 40]
Time 1713798929.3751323  Reply Finished driving 9030 S1done S2done S3done S4done S5done
Time 1713798929.3771353  Reply first: M
Time 1713798929.385135  Reply This just in ... M00000000000000000000002800c8   M   0   0   0   0   0   40   200   9032
Time 1713798929.385